In [2]:
import tensorflow as tf
import keras as K
import numpy as np

Using TensorFlow backend.


In [3]:
input = K.layers.Input(shape=(5,), name='input')
output = input
input

<tf.Tensor 'input:0' shape=(?, 5) dtype=float32>

In [6]:
model = K.Model(inputs=input, outputs=output, name='mode')
# model.summary()

x = np.array([[1,2,3,4,5], [2,3,4,5,6]])

y = np.array([1,2,3])

model.compile(loss='mean_squared_error', optimizer='sgd', mertrics=['accuracy'])

model.fit(x, y, batch_size=1, epochs=10)
# x.shape


NameError: name 'K' is not defined

In [5]:
%run evaluate.py

Using TensorFlow backend.


找不到实验的config.ini
BACKEND
backend:  resnet50
权重DEFINITIVI
正在解析标记文件
预测完成：1
class:0
gtboxes_num: 0.0
[]
precision:[]
------------------------
[none] img:000152.jpg, class:1, gt:[], det:[ 6  0 59 25  0]
[none] img:000537.jpg, class:1, gt:[], det:[355 137 429 197   0]
[none] img:000539.jpg, class:1, gt:[], det:[ 66  99 142 156   0]
[none] img:000538.jpg, class:1, gt:[], det:[254 110 329 171   0]
[wrong] img:000700.jpg, class:1, gt:[[  1  57 156  91]], det:[268  49 428  92   0]
[wrong] img:000733.jpg, class:1, gt:[[  1  62 177  98]], det:[288  52 445  95   0]
[wrong] img:000714.jpg, class:1, gt:[[  1  60 167  98]], det:[274  51 431  94   0]
[wrong] img:000695.jpg, class:1, gt:[[  1  64 129  90]], det:[238  54 397  97   0]
[wrong] img:000694.jpg, class:1, gt:[[  1  64 184 100]], det:[300  52 460  95   0]
class:1
gtboxes_num: 30.0
recall:0.0
precision:0.0
------------------------
[none] img:000100.jpg, class:2, gt:[], det:[60 63 98 84  0]
[none] img:000100.jpg, class:2, gt:[], det:[244 454 282

In [7]:
detections[:1], annotations

([[array([], shape=(0, 5), dtype=float32),
   array([], shape=(0, 5), dtype=float32),
   array([[4.3797873e+02, 1.1795082e+00, 5.1123544e+02, 8.7166748e+01,
           8.6578774e-01],
          [4.6725464e+01, 2.3156557e+02, 8.0357468e+01, 2.5457512e+02,
           3.2102582e-01]], dtype=float32),
   array([[4.6725464e+01, 2.3156557e+02, 8.0357468e+01, 2.5457512e+02,
           9.1516966e-01],
          [4.9872578e+01, 3.4798373e+02, 8.3532028e+01, 3.8389517e+02,
           8.0153340e-01],
          [9.6426117e+01, 2.6730814e+02, 1.1429962e+02, 2.8664993e+02,
           7.8671652e-01],
          [4.9577030e+01, 4.2813160e+01, 7.4391731e+01, 5.7813320e+01,
           3.6330676e-01],
          [2.2459688e+02, 4.7830099e+02, 2.5125414e+02, 4.9452469e+02,
           3.5942003e-01]], dtype=float32),
   array([], shape=(0, 5), dtype=float32),
   array([], shape=(0, 5), dtype=float32),
   array([[160.08733   ,  74.37012   , 174.3853    ,  88.283936  ,
             0.85908335],
          [210.

In [8]:
all_annotations = annotations
all_detections = detections
iou_threshold = 0.05

num_classes = len(all_annotations[0])
num_images = len(all_detections)
average_precisions = {}

# 逐个类别计算ap
for class_id in range(num_classes):

    true_positives = np.zeros((0,), dtype=np.float64)
    false_positives = np.zeros((0,), dtype=np.float64)
    scores = np.zeros((0,), dtype=np.float64)
    num_gt_boxes = 0.0

    # 记录错误
    none = []
    wrong = []

    # debug
    # if class_id != 4:
    #     continue

    # 逐个图像处理
    # print('all_detection_num:', num_images)

    for image_id in range(num_images):

        gt_boxes = all_annotations[image_id][class_id]  # (n,x1,y1,x2,y2)
        # print('cid:', class_id, 'imgid:', image_id, img_info[image_id]['filename'], len(gt_boxes))

        # 打印所有gtbox
        # print(img_info[image_id]['filename'], gt_boxes.shape[0], gt_boxes)
        num_gt_boxes += gt_boxes.shape[0]  # gt个数

        detected_gt_boxes = []  # 已经检测匹配过的gt边框

        for detect_box in all_detections[image_id][class_id]:

            scores = np.append(scores, detect_box[4])

            # 有gtbox的时候打印gt和det
            # if img_info != None:
                # print_detection = detect_box.astype(np.int)
                # print('img:{}, gt:{}, det:{}'.format(img_info[image_id], gt_boxes, print_detection))

            # 000142
            # if img_info[image_id]['filename'] == '000227.jpg':
            #     print(gt_boxes)
            #     exit()

            # 如果没有GT 边框  1！！！没有为什么要fp+ 因为没有gtbox但是被检测出来了
            if gt_boxes.shape[0] == 0:
                true_positives = np.append(true_positives, 0)
                false_positives = np.append(false_positives, 1)

                # debug
                if img_info != None:
                    img_info[image_id]['boxes'] = img_info[image_id]['boxes'][:, [1,0,3,2]]
                    # print_detection = [map(int, box) for box in enumerate(detect_box)]
                    print_gtboxes = gt_boxes.astype(np.int)
                    print_detection = detect_box.astype(np.int)
                    none.append('[none] img:{}, class:{}, gt:{}, det:{}'.format(img_info[image_id]['filename'], class_id, gt_boxes, print_detection))

                continue

            # 计算iou
            # print(gt_boxes, detect_box[:4])
            # exit()
            # if img_info[image_id]['filename'] == '000295.jpg':
            #     print(gt_boxes, detect_box)
            #     exit()

            iou = np_utils.compute_iou(gt_boxes, np.expand_dims(detect_box[:4], axis=0))  # n vs 1
            max_iou = np.max(iou, axis=0)[0]  # 与GT边框的最大iou值
            argmax_iou = np.argmax(iou, axis=0)[0]  # 最大iou值对应的GT

            # 如果超过iou阈值,且之前没有检测框匹配
            if max_iou >= iou_threshold and argmax_iou not in detected_gt_boxes:
                true_positives = np.append(true_positives, 1)
                false_positives = np.append(false_positives, 0)
                detected_gt_boxes.append(argmax_iou)
            else:
                true_positives = np.append(true_positives, 0)
                false_positives = np.append(false_positives, 1)

                # debug
                if img_info != None:
                    img_info[image_id]['boxes'] = img_info[image_id]['boxes'][:, [1,0,3,2]]
                    # print_detection = [map(int, box) for box in enumerate(detect_box)]
                    print_gtboxes = gt_boxes.astype(np.int)
                    print_detection = detect_box.astype(np.int)
                    wrong.append('[wrong] img:{}, class:{}, gt:{}, det:{}'.format(img_info[image_id]['filename'], class_id, gt_boxes, print_detection))

    if class_id != 0:
        [print(v) for k,v in enumerate(none)]
        [print(v) for k,v in enumerate(wrong)]

    # 每个类别按照得分排序
    indices = np.argsort(scores * -1)

    # [1. 1. 1. 1. 1. 0. 1. 1. 0. 0. 1. 1. 1. 0. 0. 0. 1. 0. 0.]
    true_positives = true_positives[indices]
    false_positives = false_positives[indices]
    # print(class_id, true_positives)

    # 累加 [ 1.  2.  3.  4.  5.  5.  6.  7.  7.  7.  8.  9. 10. 10. 10. 10. 11. 11. 11.]
    true_positives = np.cumsum(true_positives)
    false_positives = np.cumsum(false_positives)

    # 计算召回率和精度
    recall = true_positives / num_gt_boxes
    # print(recall)
    # print(true_positives, true_positives + false_positives)

    precision = true_positives / np.maximum(true_positives + false_positives, np.finfo(np.float64).eps)
    print('class:{}'.format(class_id))
    print('gtboxes_num: {}'.format(num_gt_boxes))
    print('recall:{}'.format(recall[-1]) if len(recall) != 0 else recall)
    # 这里preci[0]改成-1
    print('precision:{}'.format(precision[-1] if len(precision) != 0 else precision))
    print('------------------------')

    # 计算ap
#     average_precisions[class_id] = voc_ap(recall, precision, use_07_metric=use_07_metric)


class:0
gtboxes_num: 0.0
[]
precision:[]
------------------------
[none] img:000152.jpg, class:1, gt:[], det:[ 6  0 59 25  0]
[none] img:000537.jpg, class:1, gt:[], det:[355 137 429 197   0]
[none] img:000539.jpg, class:1, gt:[], det:[ 66  99 142 156   0]
[none] img:000538.jpg, class:1, gt:[], det:[254 110 329 171   0]
[wrong] img:000700.jpg, class:1, gt:[[  1  57 156  91]], det:[268  49 428  92   0]
[wrong] img:000733.jpg, class:1, gt:[[  1  62 177  98]], det:[288  52 445  95   0]
[wrong] img:000714.jpg, class:1, gt:[[  1  60 167  98]], det:[274  51 431  94   0]
[wrong] img:000695.jpg, class:1, gt:[[  1  64 129  90]], det:[238  54 397  97   0]
[wrong] img:000694.jpg, class:1, gt:[[  1  64 184 100]], det:[300  52 460  95   0]
class:1
gtboxes_num: 30.0
recall:0.0
precision:0.0
------------------------
[none] img:000100.jpg, class:2, gt:[], det:[60 63 98 84  0]
[none] img:000100.jpg, class:2, gt:[], det:[244 454 282 470   0]
[none] img:000029.jpg, class:2, gt:[], det:[121 359 159 381   0

In [43]:
print(annotations[4][6])
print(detections[4][6])
print(img_info[4]['filename'], img_info[4]['filepath'])

import matplotlib
# matplotlib.use('Agg')
# matplotlib.use('QtAgg')
print(matplotlib.get_backend())
print(matplotlib.matplotlib_fname())
from matplotlib import pyplot as plt
import cv2

img = plt.imread(img_info[4]['filepath'])
print(img.shape)
# plt.imshow(img)
# plt.show()

num = 0
for i in range(num_images):
    box = annotations[i][6]
#     print(len(box))
    num += len(box)
#     for k,v in enumerate(annotations[i][6]):
#         print(len(v))
#         num += v.shape[0]

num

[[270   1 280   7]
 [250 296 267 312]
 [303 439 315 452]
 [362 501 370 507]]
[[250.47154    295.9657     265.33337    310.94537      0.85880595]
 [301.0401     436.90427    314.656      450.34872      0.78591126]]
000024.jpg /home/speciallan/Documents/python/data/VOCdevkit/dd/JPEGImages/000024.jpg
agg
/home/speciallan/.conda/envs/py36tf/lib/python3.6/site-packages/matplotlib/mpl-data/matplotlibrc
(512, 512, 3)


284